In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# library import

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc, confusion_matrix
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.experimental import enable_hist_gradient_boosting 
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression



from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('/content/drive/MyDrive/PyTorch_YearDream/2022-01-26/data/train.csv')

# EDA
- EDA는 pr_report.html 을 참고해주세요!!


# data Split

In [6]:
X = df.drop('depvar', axis=1)
y = df['depvar']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [7]:
length = int(len(X)/5)

X_train1 = X[ : length]
X_train2 = X[length : length * 2]
X_train3 = X[length * 2 : length * 3]
X_train4 = X[length * 3 : length * 4]
X_train5 = X[length * 4 :]

y_train1 = y[ : length]
y_train2 = y[length : length * 2]
y_train3 = y[length * 2 : length * 3]
y_train4 = y[length * 3 : length * 4]
y_train5 = y[length * 4 :]


frames = [X_train1, X_train2, X_train3, X_train4] # 5
X_train_dataset1 = pd.concat(frames)

frames = [X_train1, X_train2, X_train3, X_train5] # 4
X_train_dataset2 = pd.concat(frames)

frames = [X_train1, X_train2, X_train4, X_train5] # 3
X_train_dataset3 = pd.concat(frames)

frames = [X_train1, X_train3, X_train4, X_train5] # 2
X_train_dataset4 = pd.concat(frames)

frames = [X_train2, X_train3, X_train4, X_train5] # 1
X_train_dataset5 = pd.concat(frames)


frames = [y_train1, y_train2, y_train3, y_train4] # 5
y_train_dataset1 = pd.concat(frames)

frames = [y_train1, y_train2, y_train3, y_train5] # 4
y_train_dataset2 = pd.concat(frames)

frames = [y_train1, y_train2, y_train4, y_train5] # 3
y_train_dataset3 = pd.concat(frames)

frames = [y_train1, y_train3, y_train4, y_train5] # 2
y_train_dataset4 = pd.concat(frames)

frames = [y_train2, y_train3, y_train4, y_train5] # 1
y_train_dataset5 = pd.concat(frames)

In [8]:
# f1 score와 accuracy score 두개를 고려하였습니다.
# 두 score의 값을 더해서 평균을 낸 수치가 높은값을 찾기위한 함수 정의입니다.

def calc_sum_f1_and_accuracy(y_actual, y_pred):
    return (accuracy_score(y_actual, y_pred) + f1_score(y_actual, y_pred)) / 2

In [10]:
# threshold값을 최적화 시키는 함수.
# 어떤것을 기준으로 최적화 시키는 것이냐면, 위에서 정의된 calc_sum_f1_and_accuracy()의 값이 제일 높은 경우를 찾으려고 합니다.

def calc_score_model(model, name, X_train, y_train, X_val, y_val):
    model1 = model
    model1.fit(X_train, y_train)

    y_pred1 = model1.predict(X_val)
    y_prob1 = model1.predict_proba(X_val)

    thr_result = 0.5
    max_val = 0.0

    scale = 1000
    # 반목문을 통하여 threshold값을 0.5 ~ 0.0005까지 조정하면서 calc_sum_f1_and_accuracy()값이 제일 높은 threshold값을 찾습니다.
    for thr in range(1, scale):
        val = calc_sum_f1_and_accuracy(y_val, (y_prob1[:,1] >= thr / scale))
        if val > max_val:
            thr_result = thr / scale
            max_val = val
    
    return [name, thr_result, max_val, model]



In [11]:
# method test
print(calc_score_model(XGBClassifier(), 'XGB', X_train_dataset1, y_train_dataset1, X_train5, y_train5))

['XGB', 0.362, 0.6671189032723408, XGBClassifier()]


# 각 모델별 최적화

In [ ]:
# 여기에서 사용되는 set_dict는 위에서 정의한 train data의 개수와 일치하게 만듭니다.
# 단, model의 갯수는 5개를 넘어도 상관없습니다.


set1_dict = {}
        
# name, thr, val, model = calc_score_model(XGBClassifier(), 'XGB', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
# set1_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(CatBoostClassifier(), 'Cat', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
set1_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(LGBMClassifier(), 'LGBM', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
set1_dict[name] = [val, thr, model]


name, thr, val, model = calc_score_model(RandomForestClassifier(), 'rnd', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
set1_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(DecisionTreeClassifier(), 'Decision', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
set1_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(GradientBoostingClassifier(), 'Gradient', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
set1_dict[name] = [val, thr, model]


# name, thr, val, model = calc_score_model(HistGradientBoostingClassifier(), 'Hist', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
# set1_dict[name] = [val, thr, model]

# name, thr, val, model = calc_score_model(BaggingClassifier(), 'Bag', X_train_dataset1, y_train_dataset1, X_train5, y_train5)
# set1_dict[name] = [val, thr, model]





set2_dict = {}
        
# name, thr, val, model = calc_score_model(XGBClassifier(), 'XGB', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
# set2_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(CatBoostClassifier(), 'Cat', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
set2_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(LGBMClassifier(), 'LGBM', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
set2_dict[name] = [val, thr, model]


name, thr, val, model = calc_score_model(RandomForestClassifier(), 'rnd', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
set2_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(DecisionTreeClassifier(), 'Decision', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
set2_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(GradientBoostingClassifier(), 'Gradient', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
set2_dict[name] = [val, thr, model]


# name, thr, val, model = calc_score_model(HistGradientBoostingClassifier(), 'Hist', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
# set2_dict[name] = [val, thr, model]

# name, thr, val, model = calc_score_model(BaggingClassifier(), 'Bag', X_train_dataset2, y_train_dataset2, X_train4, y_train4)
# set2_dict[name] = [val, thr, model]



set3_dict = {}
        
# name, thr, val, model = calc_score_model(XGBClassifier(), 'XGB', X_train_dataset3, y_train_dataset3, X_train3, y_train3)
# set3_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(CatBoostClassifier(), 'Cat', X_train_dataset3, y_train_dataset3, X_train3, y_train3)
set3_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(LGBMClassifier(), 'LGBM', X_train_dataset3, y_train_dataset3, X_train3, y_train3)
set3_dict[name] = [val, thr, model]


name, thr, val, model = calc_score_model(RandomForestClassifier(), 'rnd', X_train_dataset3, y_train_dataset3, X_train3, y_train3)
set3_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(DecisionTreeClassifier(), 'Decision', X_train_dataset3, y_train_dataset3, X_train3, y_train3)
set3_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(GradientBoostingClassifier(), 'Gradient', X_train_dataset3, y_train_dataset3, X_train3, y_train3)
set3_dict[name] = [val, thr, model]


# name, thr, val, model = calc_score_model(HistGradientBoostingClassifier(), 'Hist',  X_train_dataset3, y_train_dataset3, X_train3, y_train3)
# set3_dict[name] = [val, thr, model]

# name, thr, val, model = calc_score_model(BaggingClassifier(), 'Bag',  X_train_dataset3, y_train_dataset3, X_train3, y_train3)
# set3_dict[name] = [val, thr, model]




set4_dict = {}
        
# name, thr, val, model = calc_score_model(XGBClassifier(), 'XGB', X_train_dataset4, y_train_dataset4, X_train2, y_train2)
# set4_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(CatBoostClassifier(), 'Cat', X_train_dataset4, y_train_dataset4, X_train2, y_train2)
set4_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(LGBMClassifier(), 'LGBM', X_train_dataset4, y_train_dataset4, X_train2, y_train2)
set4_dict[name] = [val, thr, model]


name, thr, val, model = calc_score_model(RandomForestClassifier(), 'rnd', X_train_dataset4, y_train_dataset4, X_train2, y_train2)
set4_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(DecisionTreeClassifier(), 'Decision', X_train_dataset4, y_train_dataset4, X_train2, y_train2)
set4_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(GradientBoostingClassifier(), 'Gradient', X_train_dataset4, y_train_dataset4, X_train2, y_train2)
set4_dict[name] = [val, thr, model]


# name, thr, val, model = calc_score_model(HistGradientBoostingClassifier(), 'Hist',  X_train_dataset4, y_train_dataset4, X_train2, y_train2)
# set4_dict[name] = [val, thr, model]

# name, thr, val, model = calc_score_model(BaggingClassifier(), 'Bag',  X_train_dataset4, y_train_dataset4, X_train2, y_train2)
# set4_dict[name] = [val, thr, model]



set5_dict = {}
        
# name, thr, val, model = calc_score_model(XGBClassifier(), 'XGB', X_train_dataset5, y_train_dataset5, X_train1, y_train1)
# set5_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(CatBoostClassifier(), 'Cat', X_train_dataset5, y_train_dataset5, X_train1, y_train1)
set5_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(LGBMClassifier(), 'LGBM', X_train_dataset5, y_train_dataset5, X_train1, y_train1)
set5_dict[name] = [val, thr, model]


name, thr, val, model = calc_score_model(RandomForestClassifier(), 'rnd', X_train_dataset5, y_train_dataset5, X_train1, y_train1)
set5_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(DecisionTreeClassifier(), 'Decision', X_train_dataset5, y_train_dataset5, X_train1, y_train1)
set5_dict[name] = [val, thr, model]

name, thr, val, model = calc_score_model(GradientBoostingClassifier(), 'Gradient', X_train_dataset5, y_train_dataset5, X_train1, y_train1)
set5_dict[name] = [val, thr, model]


# name, thr, val, model = calc_score_model(HistGradientBoostingClassifier(), 'Hist',  X_train_dataset5, y_train_dataset5, X_train1, y_train1)
# set5_dict[name] = [val, thr, model]

# name, thr, val, model = calc_score_model(BaggingClassifier(), 'Bag',  X_train_dataset5, y_train_dataset5, X_train1, y_train1)
# set5_dict[name] = [val, thr, model]

In [13]:
result1 = sorted(set1_dict.items(), key=lambda x: x[1][0], reverse=True)
for k, v in result1:
    print(k, v[0], v[1])    
print("-------------")

result2 = sorted(set2_dict.items(), key=lambda x: x[1][0], reverse=True)
for k, v in result2:
    print(k, v[0], v[1])     
print("-------------")

result3 = sorted(set3_dict.items(), key=lambda x: x[1][0], reverse=True)
for k, v in result3:
    print(k, v[0], v[1])       
print("-------------")

result4 = sorted(set4_dict.items(), key=lambda x: x[1][0], reverse=True)
for k, v in result4:
    print(k, v[0], v[1])    
print("-------------")

result5 = sorted(set5_dict.items(), key=lambda x: x[1][0], reverse=True)
for k, v in result5:
    print(k, v[0], v[1])    
print("-------------")

Cat 0.687737265883217 0.363
LGBM 0.68583139448173 0.368
Gradient 0.6675123797244247 0.366
rnd 0.6484557297215036 0.381
Decision 0.5765703389830508 0.001
-------------
Cat 0.6897985839740425 0.36
LGBM 0.6868351554439782 0.342
Gradient 0.6698568019093079 0.366
rnd 0.6574241057158079 0.391
Decision 0.578490976771888 0.001
-------------
LGBM 0.6903328657056704 0.372
Cat 0.6884725601004664 0.357
Gradient 0.6671463209530484 0.335
rnd 0.6550289702233251 0.381
Decision 0.579489700770742 0.001
-------------
Cat 0.6875170129689026 0.333
LGBM 0.6852640684410647 0.391
Gradient 0.6638816281441196 0.367
rnd 0.6566285560913132 0.381
Decision 0.5780770874675623 0.001
-------------
Cat 0.6864923592493297 0.31
LGBM 0.6862856898517673 0.345
Gradient 0.668553090092721 0.349
rnd 0.6535108073505047 0.381
Decision 0.5826743902903742 0.001
-------------


In [18]:
submit = pd.read_csv('/content/drive/MyDrive/PyTorch_YearDream/2022-01-26/data/sample_submission.csv')
df_test = pd.read_csv('/content/drive/MyDrive/PyTorch_YearDream/2022-01-26/data/test.csv')
test = df_test.drop('ID', axis=1)


li = []

for i in range(len(result1)):
    prob = result1[i][1][2].predict_proba(test)
    li.append((prob[:,1] >= result1[i][1][1]).astype(np.int64))
    
    prob = result2[i][1][2].predict_proba(test)
    li.append((prob[:,1] >= result2[i][1][1]).astype(np.int64))
    
    prob = result3[i][1][2].predict_proba(test)
    li.append((prob[:,1] >= result3[i][1][1]).astype(np.int64))
    
    prob = result4[i][1][2].predict_proba(test)
    li.append((prob[:,1] >= result4[i][1][1]).astype(np.int64))
    
    prob = result5[i][1][2].predict_proba(test)
    li.append((prob[:,1] >= result5[i][1][1]).astype(np.int64))




1

In [19]:
# 전체 모델의 갯수
len(li)

25

In [20]:
# 과반이상의 갯수를 출력
round(len(li)/2+0.51)

13

In [ ]:
result = []
for i in range(len(test)):
    score = 0
    for j in range(len(li)):
        score += li[j][i]
    if round(len(li)/2+0.51) <= score:
        result.append(1)
    else:
        result.append(0)

print(f" 최종 결과물!!! ")
print(f" {sum(result) / len(result)} ")
print(len(result))
print(result.count(1))

submit["answer"] = result  

# 제출 파일 저장
submit.to_csv('/content/drive/MyDrive/PyTorch_YearDream/2022-01-26/submission/prediction.csv', index=False)

 최종 결과물!!! 
 0.36709850346213985 
35816
13148


# 정리하자면..

<ul>
<li>제공받은 data를 5개로 나눕니다.</li>
<li>이제 학습시킬 모델을 선정합니다.(여기에서는 Tree기반 모델 5개를 골랐으며, hyperparameter tuning을 진행하지 않았습니다. feature enginering이 잘 되어있어서, 괜찮은 성능을 보여준다고 생각했습니다.)</li>
<li>선정한 모델들을 dictionary에 담고 dictionary의 갯수는 data split한 갯수와 동일해야 합니다.</li>
<li>학습을 시키는데, dictionary마다 다른 데이터셋을 학습시킵니다.<br/>
dictionary 1번은 data1,data2,data3,data4 	&nbsp;	&nbsp; --> 5번 데이터 제외<br/>
dictionary 2번은 data1,data2,data3,data5 	&nbsp;	&nbsp; --> 4번 데이터 제외<br/>
dictionary 3번은 data1,data2,data4,data5 	&nbsp;	&nbsp; --> 3번 데이터 제외<br/>
dictionary 4번은 data1,data3,data4,data5 	&nbsp;	&nbsp; --> 2번 데이터 제외<br/>
dictionary 5번은 data2,data3,data4,data5 	&nbsp;	&nbsp; --> 1번 데이터 제외<br/></li>
<li>위와 같이 학습을 시킨다면 장점으로는 원래는 데이터가 10만개 뿐이였지만, 지금은 5등분으로 5개의 셋을 만들어내어, 40만개의 데이터셋의 학습효과를 기대해 볼 수 있습니다.</li>
<li>마지막으로 성능을 더 끌어올리기 위하여, hard voting을 실시합니다.<br/>
여기서는 그냥 과반이 넘기만 하면, 과반을 넘은 클래스로 최종결과를 제출하는 방법을 택했지만, 전체의 70% 또는 80%가 동일한 클래스로 나와야지만 제출하는 방법도 있습니다.</li>
<li></li>
<li></li>
</ul>